In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")

In [3]:
df = df_.copy()

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [5]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [6]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [7]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,515255.00000,10.95663,-9600.00000,1.00000,3.00000,10.00000,19152.00000,104.35401
InvoiceDate,515255,2010-06-28 17:41:06.558422528,2009-12-01 07:45:00,2010-03-21 13:27:00,2010-07-06 13:13:00,2010-10-15 14:27:00,2010-12-09 20:01:00,NaN
Price,515255.00000,3.95637,-53594.36000,1.25000,2.10000,4.21000,25111.09000,127.68856
Customer ID,407695.00000,15368.50411,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000,1679.79570


In [8]:
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)

In [9]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [10]:
cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})

In [11]:
cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

#### 2. Average Order Value (average_order_value = total_price / total_transaction)

In [12]:
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


In [13]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

#### 3. Purchase Frequency (total_transaction / total_number_of_customers)

In [14]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.00000,11,70,372.86000,33.89636
12347.00000,2,828,1323.32000,661.66000
12348.00000,1,373,222.16000,222.16000
12349.00000,3,993,2671.14000,890.38000
12351.00000,1,261,300.93000,300.93000


In [15]:
cltv_c.shape[0]

4314

In [16]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

#### 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)

In [17]:
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]

In [18]:
churn_rate = 1 - repeat_rate

#### 5. Profit Margin (profit_margin =  total_price * 0.10)

In [19]:
cltv_c['profit_margin'] = cltv_c['total_price'] * 0.10

#### 6. Customer Value (customer_value = average_order_value * purchase_frequency)

In [20]:
cltv_c['customer_value'] = cltv_c['average_order_value'] * cltv_c["purchase_frequency"]

#### 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)

In [21]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


#### 8. Segmentlerin Oluşturulması

In [22]:
cltv_c.sort_values(by="cltv", ascending=False).tail()

cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

cltv_c.sort_values(by="cltv", ascending=False).head()

cltv_c.groupby("segment").agg({"count", "mean", "sum"})

cltv_c.to_csv("cltc_c.csv")

# 18102.00000       A
# 14646.00000       A
# 14156.00000       A
# 14911.00000       A
# 13694.00000       A

# Customer ID
# 18102.00000       A
# 14646.00000       A
# 14156.00000       A
# 14911.00000       A
# 13694.00000       A

C:\Users\derya\AppData\Local\Temp\ipykernel_15740\481795899.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cltv_c.groupby("segment").agg({"count", "mean", "sum"})


#### 9. BONUS: Tüm İşlemlerin Fonksiyonlaştırılması

In [23]:
def create_cltv_c(dataframe, profit=0.10):

    # Veriyi hazırlama
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[(dataframe['Quantity'] > 0)]
    dataframe.dropna(inplace=True)
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                                   'Quantity': lambda x: x.sum(),
                                                   'TotalPrice': lambda x: x.sum()})
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    # avg_order_value
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    # purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    # repeat rate & churn rate
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    # profit_margin
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    # Customer Value
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    # Customer Lifetime Value
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    # Segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_c


df = df_.copy()

clv = create_cltv_c(df)

#### The End